# 3D detailed analysis

This tutorial does a 3D map based analsis on the galactic center, using simulated observations from the CTA-1DC. We will use the high level interface for the data reduction, and then do a detailed modelling. This will be done in two different ways:

- stacking all the maps together and fitting the stacked maps
- handling all the observations separately and doing a joint fitting on all the maps

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from pathlib import Path
from regions import CircleSkyRegion
from scipy.stats import norm
from gammapy.analysis import Analysis, AnalysisConfig
from gammapy.modeling.models import (
    SkyModel,
    ExpCutoffPowerLawSpectralModel,
    PointSpatialModel,
    FoVBackgroundModel,
    Models,
)
from gammapy.modeling import Fit
from gammapy.maps import Map
from gammapy.estimators import ExcessMapEstimator
from gammapy.datasets import MapDataset

## Analysis configuration

In this section we select observations and define the analysis geometries, irrespective of  joint/stacked analysis. For configuration of the analysis, we will programatically build a config file from scratch.

In [2]:
config = AnalysisConfig()
# The config file is now empty, with only a few defaults specified.
print(config)

AnalysisConfig

    general:
        log: {level: info, filename: null, filemode: null, format: null, datefmt: null}
        outdir: .
    observations:
        datastore: $GAMMAPY_DATA/hess-dl3-dr1
        obs_ids: []
        obs_file: null
        obs_cone: {frame: null, lon: null, lat: null, radius: null}
        obs_time: {start: null, stop: null}
        required_irf: [aeff, edisp, psf, bkg]
    datasets:
        type: 1d
        stack: true
        geom:
            wcs:
                skydir: {frame: null, lon: null, lat: null}
                binsize: 0.02 deg
                width: {width: 5.0 deg, height: 5.0 deg}
                binsize_irf: 0.2 deg
            selection: {offset_max: 2.5 deg}
            axes:
                energy: {min: 1.0 TeV, max: 10.0 TeV, nbins: 5}
                energy_true: {min: 0.5 TeV, max: 20.0 TeV, nbins: 16}
        map_selection: [counts, exposure, background, psf, edisp]
        background:
            method: null
            exclusion:

In [3]:
# Selecting the observations
config.observations.datastore = "$GAMMAPY_DATA/cta-1dc/index/gps/"
config.observations.obs_ids = [110380, 111140, 111159]

In [4]:
# Defining a reference geometry for the reduced datasets

config.datasets.type = "3d"  # Analysis type is 3D

config.datasets.geom.wcs.skydir = {
    "lon": "0 deg",
    "lat": "0 deg",
    "frame": "galactic",
}  # The WCS geometry - centered on the galactic center
config.datasets.geom.wcs.width = {"width": "10 deg", "height": "8 deg"}
config.datasets.geom.wcs.binsize = "0.02 deg"

# Cutout size (for the run-wise event selection)
config.datasets.geom.selection.offset_max = 3.5 * u.deg
config.datasets.safe_mask.methods = ["aeff-default", "offset-max"]

# We now fix the energy axis for the counts map - (the reconstructed energy binning)
config.datasets.geom.axes.energy.min = "0.1 TeV"
config.datasets.geom.axes.energy.max = "10 TeV"
config.datasets.geom.axes.energy.nbins = 10

# We now fix the energy axis for the IRF maps (exposure, etc) - (the true enery binning)
config.datasets.geom.axes.energy_true.min = "0.08 TeV"
config.datasets.geom.axes.energy_true.max = "12 TeV"
config.datasets.geom.axes.energy_true.nbins = 14

In [5]:
print(config)

AnalysisConfig

    general:
        log: {level: info, filename: null, filemode: null, format: null, datefmt: null}
        outdir: .
    observations:
        datastore: $GAMMAPY_DATA/cta-1dc/index/gps
        obs_ids: [110380, 111140, 111159]
        obs_file: null
        obs_cone: {frame: null, lon: null, lat: null, radius: null}
        obs_time: {start: null, stop: null}
        required_irf: [aeff, edisp, psf, bkg]
    datasets:
        type: 3d
        stack: true
        geom:
            wcs:
                skydir: {frame: galactic, lon: 0.0 deg, lat: 0.0 deg}
                binsize: 0.02 deg
                width: {width: 10.0 deg, height: 8.0 deg}
                binsize_irf: 0.2 deg
            selection: {offset_max: 3.5 deg}
            axes:
                energy: {min: 0.1 TeV, max: 10.0 TeV, nbins: 10}
                energy_true: {min: 0.08 TeV, max: 12.0 TeV, nbins: 14}
        map_selection: [counts, exposure, background, psf, edisp]
        background:
       

## Configuration for stacked and joint analysis

This is done just by specfiying the flag on `config.datasets.stack`. Since the internal machinery will work differently for the two cases, we will write it as two config files and save it to disc in YAML format for future reference. 

In [6]:
config_stack = config.copy(deep=True)
config_stack.datasets.stack = True

config_joint = config.copy(deep=True)
config_joint.datasets.stack = False

In [7]:
# To prevent unnecessary cluttering, we write it in a separate folder.
path = Path("analysis_3d")
path.mkdir(exist_ok=True)
config_joint.write(path=path / "config_joint.yaml", overwrite=True)
config_stack.write(path=path / "config_stack.yaml", overwrite=True)

## Stacked analysis

### Data reduction

We first show the steps for the stacked analysis and then repeat the same for the joint analysis later


In [8]:
# Reading yaml file:
config_stacked = AnalysisConfig.read(path=path / "config_stack.yaml")

In [9]:
analysis_stacked = Analysis(config_stacked)

Setting logging config: {'level': 'INFO', 'filename': None, 'filemode': None, 'format': None, 'datefmt': None}


In [10]:
%%time
# select observations:
analysis_stacked.get_observations()

# run data reduction
analysis_stacked.get_datasets()

Fetching observations.
Number of selected observations: 3
Creating reference dataset and makers.
Creating the background Maker.
No background maker set. Check configuration.
Start the data reduction loop.
Invalid unit found in background table! Assuming (s-1 MeV-1 sr-1)
Invalid unit found in background table! Assuming (s-1 MeV-1 sr-1)
No default thresholds defined for obs 110380
Invalid unit found in background table! Assuming (s-1 MeV-1 sr-1)
Invalid unit found in background table! Assuming (s-1 MeV-1 sr-1)
No default thresholds defined for obs 111140
Invalid unit found in background table! Assuming (s-1 MeV-1 sr-1)
Invalid unit found in background table! Assuming (s-1 MeV-1 sr-1)
No default thresholds defined for obs 111159


CPU times: user 8.28 s, sys: 1.62 s, total: 9.89 s
Wall time: 9.89 s


In [11]:
dataset_stacked = analysis_stacked.datasets["stacked"]
print(dataset_stacked)

MapDataset
----------

  Name                            : stacked 

  Total counts                    : 121241 
  Total background counts         : 108043.52
  Total excess counts             : 13197.48

  Predicted counts                : 108043.52
  Predicted background counts     : 108043.52
  Predicted excess counts         : nan

  Exposure min                    : 6.28e+07 m2 s
  Exposure max                    : 1.90e+10 m2 s

  Number of total bins            : 2000000 
  Number of fit bins              : 1411180 

  Fit statistic type              : cash
  Fit statistic value (-2 log(L)) : nan

  Number of models                : 0 
  Number of parameters            : 0
  Number of free parameters       : 0




### Modeling and fitting

Now comes the interesting part of the analysis - choosing appropriate models for our source and fitting them.

We choose a point source model with an exponential cutoff power-law spectrum.

In [12]:
dataset_stacked.mask_fit = dataset_stacked.counts.geom.energy_mask(energy_min=0.3*u.TeV,
                            energy_max=None)

In [13]:
bkg_model = FoVBackgroundModel(dataset_name="stacked")
bkg_model.spectral_model.norm.value = 1.3

models_stacked = Models([bkg_model])
dataset_stacked.models = models_stacked

In [14]:
%%time
fit = Fit(optimize_opts={"print_level": 1})
result = fit.run(datasets=[dataset_stacked])

------------------------------------------------------------------
| FCN = 1.817e+05               |      Ncalls=21 (21 total)      |
| EDM = 1.77e-07 (Goal: 0.0002) |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   True    |   True    | False  |
------------------------------------------------------------------
CPU times: user 1.11 s, sys: 171 ms, total: 1.28 s
Wall time: 1.29 s


In [15]:
L0 = dataset_stacked.stat_sum()

In [16]:
models_stacked.parameters.to_table().pprint_all()

  type      name     value    unit   error   min max frozen
-------- --------- ---------- ---- --------- --- --- ------
spectral      norm 1.3678e+00      9.373e-03 nan nan  False
spectral      tilt 0.0000e+00      0.000e+00 nan nan   True
spectral reference 1.0000e+00  TeV 0.000e+00 nan nan   True


In [17]:
spatial_model = PointSpatialModel(
    lon_0="-0.05 deg", lat_0="-0.05 deg", frame="galactic"
)
spectral_model = ExpCutoffPowerLawSpectralModel(
    index=2.3,
    amplitude=2.8e-12 * u.Unit("cm-2 s-1 TeV-1"),
    reference=1.0 * u.TeV,
    lambda_=0.02 / u.TeV,
)

model = SkyModel(
    spatial_model=spatial_model,
    spectral_model=spectral_model,
    name="gc-source",
)


models_stacked = Models([model, bkg_model])

dataset_stacked.models = models_stacked

In [18]:
%%time
fit = Fit(optimize_opts={"print_level": 1})
result = fit.run(datasets=[dataset_stacked])

------------------------------------------------------------------
| FCN = 1.805e+05               |     Ncalls=190 (190 total)     |
| EDM = 3.36e-05 (Goal: 0.0002) |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   True    |   True    | False  |
------------------------------------------------------------------
CPU times: user 12.8 s, sys: 1.48 s, total: 14.2 s
Wall time: 14.4 s


In [19]:
L1 = dataset_stacked.stat_sum()
del_TS = L0-L1
print(del_TS)

1284.9229974735354


In [20]:
models_stacked.parameters.to_table().pprint_all()

  type      name      value         unit        error      min        max    frozen
-------- --------- ----------- -------------- --------- ---------- --------- ------
spectral     index  2.4141e+00                1.523e-01        nan       nan  False
spectral amplitude  2.6635e-12 cm-2 s-1 TeV-1 3.104e-13        nan       nan  False
spectral reference  1.0000e+00            TeV 0.000e+00        nan       nan   True
spectral   lambda_ -1.3283e-02          TeV-1 6.840e-02        nan       nan  False
spectral     alpha  1.0000e+00                0.000e+00        nan       nan   True
 spatial     lon_0 -4.8064e-02            deg 2.597e-03        nan       nan  False
 spatial     lat_0 -5.2606e-02            deg 2.256e-03 -9.000e+01 9.000e+01  False
spectral      norm  1.3481e+00                9.314e-03        nan       nan  False
spectral      tilt  0.0000e+00                0.000e+00        nan       nan   True
spectral reference  1.0000e+00            TeV 0.000e+00        nan       nan

## Estimators

In [21]:
from gammapy.estimators import FluxPointsEstimator

In [22]:
e_min, e_max = 0.1, 10
energy_edges = np.geomspace(e_min, e_max, 11) * u.TeV
energy_edges

<Quantity [ 0.1       ,  0.15848932,  0.25118864,  0.39810717,  0.63095734,
            1.        ,  1.58489319,  2.51188643,  3.98107171,  6.30957344,
           10.        ] TeV>

### Reoptimize = false

In [23]:
fpe = FluxPointsEstimator(energy_edges=energy_edges, source='gc-source')
flux_points = fpe.run(dataset_stacked)

In [24]:
flux_points.ts.data

array([[[         nan]],

       [[         nan]],

       [[         nan]],

       [[273.48086524]],

       [[215.86683331]],

       [[184.29113013]],

       [[241.17497767]],

       [[147.29497937]],

       [[146.38933979]],

       [[ 84.00263254]]])

In [25]:
np.nansum(flux_points.ts.data), del_TS

(1292.5007580424972, 1284.9229974735354)

### Reoptimize = true

In [26]:
fpe1 = FluxPointsEstimator(energy_edges=energy_edges, source='gc-source', reoptimize=True)
flux_points1 = fpe1.run(dataset_stacked)

Position <SkyCoord (Galactic): (l, b) in deg
    (6.43293418, -0.0434748)> is outside valid IRF map range, using nearest IRF defined within
Position <SkyCoord (Galactic): (l, b) in deg
    (6.43293418, -0.0434748)> is outside valid IRF map range, using nearest IRF defined within
Position <SkyCoord (Galactic): (l, b) in deg
    (353.46943944, -0.0434748)> is outside valid IRF map range, using nearest IRF defined within
Position <SkyCoord (Galactic): (l, b) in deg
    (353.46943944, -0.0434748)> is outside valid IRF map range, using nearest IRF defined within
Position <SkyCoord (Galactic): (l, b) in deg
    (64.76866052, -0.0434748)> is outside valid IRF map range, using nearest IRF defined within
Position <SkyCoord (Galactic): (l, b) in deg
    (64.76866052, -0.0434748)> is outside valid IRF map range, using nearest IRF defined within
Position <SkyCoord (Galactic): (l, b) in deg
    (295.13371311, -0.0434748)> is outside valid IRF map range, using nearest IRF defined within
Position <Sk

Position <SkyCoord (Galactic): (l, b) in deg
    (16.29137107, -0.05995728)> is outside valid IRF map range, using nearest IRF defined within
Position <SkyCoord (Galactic): (l, b) in deg
    (16.29137107, -0.05995728)> is outside valid IRF map range, using nearest IRF defined within
Position <SkyCoord (Galactic): (l, b) in deg
    (38.09729211, -0.05599332)> is outside valid IRF map range, using nearest IRF defined within
Position <SkyCoord (Galactic): (l, b) in deg
    (38.09729211, -0.05599332)> is outside valid IRF map range, using nearest IRF defined within
Position <SkyCoord (Galactic): (l, b) in deg
    (321.8241086, -0.05599332)> is outside valid IRF map range, using nearest IRF defined within
Position <SkyCoord (Galactic): (l, b) in deg
    (321.8241086, -0.05599332)> is outside valid IRF map range, using nearest IRF defined within
Position <SkyCoord (Galactic): (l, b) in deg
    (21.32661792, -0.05599332)> is outside valid IRF map range, using nearest IRF defined within
Positi

In [27]:
np.nansum(flux_points1.ts.data), del_TS

(1263.176156789761, 1284.9229974735354)

### different energy range

In [28]:
e_min, e_max = 0.4, 10
energy_edges = np.geomspace(e_min, e_max, 11) * u.TeV
energy_edges

<Quantity [ 0.4       ,  0.55189186,  0.76146158,  1.05061112,  1.44955933,
            2.        ,  2.75945932,  3.80730788,  5.25305561,  7.24779664,
           10.        ] TeV>

In [29]:
fpe2 = FluxPointsEstimator(energy_edges=energy_edges, source='gc-source')
flux_points2 = fpe2.run(dataset_stacked)

Dataset stacked does not contribute in the energy range


ValueError: 'gc-source' is not in list